In [59]:
# load packages
import numpy as np
import scipy as sp
import pandas as pd
from scipy import optimize as opt
from scipy.integrate import odeint 
from scipy.optimize import minimize

Now consider parameter learning for the SuEIR model. Given the model parameters $\boldsymbol{\theta}$ and initial quantities $S_0, E_0, I_0$, and $R_0$, we can compute the number of individuals in each group (i.e., $S, E, I$, and $R$ ) at time $t$, denoted by $\widehat{S}_t, \widehat{E}_t, \widehat{I}_t$ and $\widehat{R}_t$, via applying  numerical ODE solvers onto the ODE. Then we propose to learn the model parameter $\widehat{\boldsymbol{\theta}}=(\widehat{\beta}, \widehat{\sigma}, \widehat{\gamma}, \widehat{\mu})$ by minimizing the following logarithmic-type mean square error (MSE).

Where $\mathbf{I}=\left\{I_t\right\}_{t=1}^T, \mathbf{R}=\left\{R_t\right\}_{t=1}^T$ with $I_t$ and $R_t$ denote the reported numbers of infected cases and removed cases (including both recovered cases and fatality cases) at time $t$ (i.e., date), and $p$ is the smoothing parameter used to ensure numerical stability. Note that given $S_0, E_0, I_0$ and $R_0, \widehat{I}_t$ and $\widehat{R}_t$ can be described as differentiable functions of the parameter $\boldsymbol{\theta}$. Then the model parameter $\widehat{\boldsymbol{\theta}}=\operatorname{argmin}_{\boldsymbol{\theta}} L(\boldsymbol{\theta} ; \mathbf{I}, \mathbf{R})$ can be learnt by applying standard gradient based optimizer (e.g., BFGS) onto the loss function under the constraint that $\beta, \sigma, \gamma, \mu \in[0,1]$.

$$
L(\boldsymbol{\theta} ; \mathbf{I}, \mathbf{R})=\frac{1}{T} \sum_{t=1}^T\left[\left(\log \left(\widehat{I}_t+p\right)-\log \left(I_t+p\right)\right)^2+\left(\log \left(\widehat{R}_t+p\right)-\log \left(R_t+p\right)\right)^2\right],
$$


In [60]:
def loss(pred, target, smoothing=20): 
    return np.mean((np.log(pred+smoothing) - np.log(target+smoothing))**2)
    
# SEIR: the 'reported' SEIR. In our case, the simulated dataset. 
def optim(SEIR, method = 'L-BFGS-B', init= [0.15, 0.075, 0.05, 0.1]):  #beta, mu, sigma, gamma
    # maybe these could be passed into the function as  
    # parameters if the estimation parts change
    y0 = [800.0, 100.0, 50.0, 50.0]
    t = np.linspace(0, 59, 60)
    N  = 1000.0  
    def deriv(y, t, N, beta, mu, sigma, gamma):
        s,e,i,r= y
        dsdt = -((beta * (e + i) * s) / N)
        dedt = (((beta * (e + i) * s) / N) - sigma * e)
        didt = (mu * sigma * e - (gamma * i))
        drdt = (gamma * i)  
        return dsdt, dedt, didt, drdt
    
    def objective(x):
        beta, mu, sigma, gamma = x
        est_SEIR = odeint(deriv, y0, t, args=(N, beta, mu, sigma, gamma))
        est_df = pd.DataFrame(est_SEIR, columns = ['S','E','I','R'])
        return loss(est_df.loc[:,"I"], SEIR.loc[:,"I"]) +  loss(est_df.loc[:,"R"], SEIR.loc[:,"R"])

    # scipy optimizer
    optimal = minimize(
        objective,
        init, # initial estimate
        method=method,  
        bounds=[(0.0001, 1),  (0.001, 1.), (0.01, 1), (0.05, 0.3)] # bounded based on prior knowledge: beta, mu, sigma, gamma
        # bounds=[(0, 1),  (0, 1), (0,1), (0, 1)]
    )

    return optimal.x

Load datasets and then minimize the objective function and obtain parameter estimates:

In [61]:
os.chdir('c:\\Users\\chloe\\Desktop\\ChloeYou\\UBC_grad\\Term3\\MATH561\\math561-gp-SuEIR\\data\\python')

params_nn = pd.DataFrame([])
for file in os.listdir(os.getcwd()) :
    df = pd.read_csv(file)
    params_nn[file] = optim(df)
    
params_nn # rows: beta, mu, sigma, gamma


,sim-1-p.csv,sim-10-p.csv,sim-2-p.csv,sim-3-p.csv,sim-4-p.csv,sim-5-p.csv,sim-6-p.csv,sim-7-p.csv,sim-8-p.csv,sim-9-p.csv
0,0.570735,0.055112,0.066090,0.061090,0.232100,0.085348,0.381025,0.397532,0.070860,0.055911
1,0.067201,0.973298,1.000000,0.707741,0.134965,0.424742,0.094986,0.092907,0.557682,1.000000
2,0.193610,0.037506,0.038232,0.049651,0.124355,0.072149,0.149208,0.153206,0.055559,0.036574
3,0.128469,0.120997,0.119917,0.120778,0.121669,0.120812,0.123563,0.124567,0.119964,0.120454


In [62]:
os.chdir('c:\\Users\\chloe\\Desktop\\ChloeYou\\UBC_grad\\Term3\\MATH561\\math561-gp-SuEIR\\data\\matlab')

csv = os.listdir(os.getcwd())
del csv[0] # this is the ground truth so we will only calculate the R_0 from it
print(csv)
params_trad = pd.DataFrame([])
for file in csv :
    df = pd.read_csv(file)
    params_trad[file] = optim(df)
    
params_trad # rows: beta, mu, sigma, gamma

['sim-1-m.csv', 'sim-10-m.csv', 'sim-2-m.csv', 'sim-3-m.csv', 'sim-4-m.csv', 'sim-5-m.csv', 'sim-6-m.csv', 'sim-7-m.csv', 'sim-8-m.csv', 'sim-9-m.csv']


,sim-1-m.csv,sim-10-m.csv,sim-2-m.csv,sim-3-m.csv,sim-4-m.csv,sim-5-m.csv,sim-6-m.csv,sim-7-m.csv,sim-8-m.csv,sim-9-m.csv
0,0.333303,0.100104,0.399304,0.299962,0.200065,0.100190,0.328968,0.399410,0.299859,0.200017
1,0.055199,0.099911,0.050012,0.050006,0.049987,0.049917,0.109642,0.100016,0.100017,0.099992
2,0.145983,0.090037,0.090202,0.090001,0.089968,0.090073,0.149128,0.090171,0.090051,0.089987
3,0.119583,0.120001,0.120000,0.120003,0.120001,0.120001,0.119403,0.120000,0.120001,0.119997


Calculate R_0 for each simulation:
$$
R_0 = \frac{\beta}{\sigma} +\frac{\beta\mu}{\gamma}
$$

In [63]:
R_0_trad = (params_trad.loc[0,]/params_trad.loc[2,]) + (params_trad.loc[0,]*params_trad.loc[1,]/params_trad.loc[3,])
R_0_trad


sim-1-m.csv     2.437009
sim-10-m.csv    1.195149
sim-2-m.csv     4.593166
sim-3-m.csv     3.457847
sim-4-m.csv     2.307090
sim-5-m.csv     1.153991
sim-6-m.csv     2.508014
sim-7-m.csv     4.762369
sim-8-m.csv     3.579801
sim-9-m.csv     2.389395
dtype: float64

In [64]:
R_0_nn = (params_nn.loc[0,]/params_nn.loc[2,]) + (params_nn.loc[0,]*params_nn.loc[1,]/params_nn.loc[3,])
R_0_nn

sim-1-p.csv     3.246414
sim-10-p.csv    1.912754
sim-2-p.csv     2.279809
sim-3-p.csv     1.588377
sim-4-p.csv     2.123892
sim-5-p.csv     1.482994
sim-6-p.csv     2.846553
sim-7-p.csv     2.891238
sim-8-p.csv     1.604810
sim-9-p.csv     1.992853
dtype: float64

Let's compare these estimated values to the 'true' $R_0 = \frac{0.25}{0.09} +\frac{0.25*0.075}{0.12} = 2.9$ 

Also, since the objective function is non-convex, the initial parameter used for optimization
varies the estimate a lot. 

We look at the sum of the absolute value of bias:

#### Testing different optimization algorithms and initial parameter estimates 

In [65]:
os.chdir('c:\\Users\\chloe\\Desktop\\ChloeYou\\UBC_grad\\Term3\\MATH561\\math561-gp-SuEIR\\data\\matlab')

csv = os.listdir(os.getcwd())
del csv[0] # this is the ground truth so we will only calculate the R_0 from it
print(csv)

params_trad = pd.DataFrame([])
for file in csv :
    df = pd.read_csv(file)
    params_trad[file] = optim(df, method='Powell', init=[0.5,0.5,0.5,0.05])
    
params_trad # rows: beta, mu, sigma, gamma

R_0_trad = (params_trad.loc[0,]/params_trad.loc[2,]) + (params_trad.loc[0,]*params_trad.loc[1,]/params_trad.loc[3,])
R_0_trad

['sim-1-m.csv', 'sim-10-m.csv', 'sim-2-m.csv', 'sim-3-m.csv', 'sim-4-m.csv', 'sim-5-m.csv', 'sim-6-m.csv', 'sim-7-m.csv', 'sim-8-m.csv', 'sim-9-m.csv']


sim-1-m.csv     1.017865
sim-10-m.csv    0.996793
sim-2-m.csv     5.268347
sim-3-m.csv     3.458939
sim-4-m.csv     2.307064
sim-5-m.csv     1.042159
sim-6-m.csv     0.282606
sim-7-m.csv     0.337580
sim-8-m.csv     0.448378
sim-9-m.csv     1.008896
dtype: float64

In [66]:
params_trad = pd.DataFrame([])
for file in csv :
    df = pd.read_csv(file)
    params_trad[file] = optim(df, method='Powell', init=[0.15, 0.075, 0.05, 0.1])
    
params_trad # rows: beta, mu, sigma, gamma

R_0_trad = (params_trad.loc[0,]/params_trad.loc[2,]) + (params_trad.loc[0,]*params_trad.loc[1,]/params_trad.loc[3,])
R_0_trad

sim-1-m.csv     2.616884
sim-10-m.csv    1.195837
sim-2-m.csv     4.592966
sim-3-m.csv     1.032465
sim-4-m.csv     2.307061
sim-5-m.csv     0.929623
sim-6-m.csv     2.508438
sim-7-m.csv     4.762620
sim-8-m.csv     3.580645
sim-9-m.csv     7.844318
dtype: float64

In [67]:
os.chdir('c:\\Users\\chloe\\Desktop\\ChloeYou\\UBC_grad\\Term3\\MATH561\\math561-gp-SuEIR\\data\\python')

params_nn = pd.DataFrame([])
for file in os.listdir(os.getcwd()):
    df = pd.read_csv(file)
    params_nn[file] = optim(df,method='Powell', init=[0.5,0.5,0.5,0.05])
    
params_nn # rows: beta, mu, sigma, gamma

R_0_nn = (params_nn.loc[0,]/params_nn.loc[2,]) + (params_nn.loc[0,]*params_nn.loc[1,]/params_nn.loc[3,])
R_0_nn


sim-1-p.csv     3.249635
sim-10-p.csv    1.913048
sim-2-p.csv     2.289714
sim-3-p.csv     1.588414
sim-4-p.csv     0.358568
sim-5-p.csv     1.520930
sim-6-p.csv     0.002193
sim-7-p.csv     0.002297
sim-8-p.csv     1.926397
sim-9-p.csv     2.003549
dtype: float64

In [68]:
params_nn = pd.DataFrame([])
for file in os.listdir(os.getcwd()):
    df = pd.read_csv(file)
    params_nn[file] = optim(df,method='Powell', init=[0.15, 0.075, 0.05, 0.1])
    
params_nn # rows: beta, mu, sigma, gamma

R_0_nn = (params_nn.loc[0,]/params_nn.loc[2,]) + (params_nn.loc[0,]*params_nn.loc[1,]/params_nn.loc[3,])
R_0_nn


sim-1-p.csv     3.280390
sim-10-p.csv    1.478755
sim-2-p.csv     1.637985
sim-3-p.csv     1.394264
sim-4-p.csv     1.227441
sim-5-p.csv     1.574160
sim-6-p.csv     2.839733
sim-7-p.csv     2.890974
sim-8-p.csv     1.238996
sim-9-p.csv     1.502232
dtype: float64